In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, initializers, backend as K
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import shutil

In [2]:
# Configuration
INPUT_SHAPE = (224, 224, 3)
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.001
TEST_SIZE = 0.2

In [3]:
def train_initial_model():
    train_dir = r"D:\Major Project\Rasp\old\initial_train"
    test_dir = r"D:\Major Project\Rasp\old\test"
    
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )
    
    train_gen = train_datagen.flow_from_directory(
        train_dir,
        target_size=INPUT_SHAPE[:2],
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='training',
        shuffle=True
    )
    
    val_gen = train_datagen.flow_from_directory(
        train_dir,
        target_size=INPUT_SHAPE[:2],
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='validation'
    )

    base_model = tf.keras.applications.VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=INPUT_SHAPE
    )
    base_model.trainable = False  # freeze base model
    
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')  # binary classification
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=EPOCHS,
        verbose=1
    )
    
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_gen = test_datagen.flow_from_directory(
        test_dir,
        target_size=INPUT_SHAPE[:2],
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )
    
    y_pred_probs = model.predict(test_gen)
    y_pred = (y_pred_probs > 0.5).astype(int).reshape(-1)

    y_true = test_gen.classes
    
    print("\nTest Metrics:")
    print(f"Accuracy: {np.mean(y_true == y_pred):.4f}")
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=['Not Drowsy', 'Drowsy']))
    
   # Inside train_initial_model() function after training:
    model.save(r"D:\Major Project\Report\model_waste\vgg_20_epoch.keras")  

    return model

In [4]:
train_initial_model()

Found 9019 images belonging to 2 classes.
Found 2253 images belonging to 2 classes.


D:\Major Project\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1175s 4s/step - accuracy: 0.6633 - loss: 0.7917 - val_accuracy: 0.6028 - val_loss: 0.9246
Epoch 2/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1165s 4s/step - accuracy: 0.8368 - loss: 0.3440 - val_accuracy: 0.5517 - val_loss: 1.2835
Epoch 3/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1160s 4s/step - accuracy: 0.8508 - loss: 0.3064 - val_accuracy: 0.5424 - val_loss: 1.3195
Epoch 4/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1171s 4s/step - accuracy: 0.8578 - loss: 0.2760 - val_accuracy: 0.5517 - val_loss: 1.5551
Epoch 5/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1164s 4s/step - accuracy: 0.8662 - loss: 0.2659 - val_accuracy: 0.5348 - val_loss: 2.4296
Epoch 6/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1163s 4s/step - accuracy: 0.8759 - loss: 0.2418 - val_accuracy: 0.5490 - val_loss: 2.5931
Epoch 7/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1167s 4s/step - accuracy: 0.8823 - loss: 0.2362 - val_accuracy: 0.5499 - val_loss: 2.9212
Epoch 8/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1166s 4s/step - accuracy: 0.8760 - loss: 0.2314 - 

<Sequential name=sequential, built=True>